#Logistics

In [4]:
from google.colab import auth
auth.authenticate_user()

print('Authenticated')

Authenticated


In [5]:
project_id = 'bigquerycourse-464720'

In [6]:
%load_ext google.colab.data_table

#Deliveries in within 48 hours?

> For this query we will use the orders_item table

In [7]:
%%bigquery --project {project_id}
select
*
from
  `bigquery-public-data.thelook_ecommerce.order_items`
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,id,order_id,user_id,product_id,inventory_item_id,status,created_at,shipped_at,delivered_at,returned_at,sale_price
0,4647,3230,2571,14235,12563,Complete,2024-07-28 08:22:13+00:00,2024-07-30 19:07:00+00:00,2024-08-02 06:57:00+00:00,NaT,0.02
1,5665,3919,3115,14235,15362,Complete,2025-01-05 16:02:20+00:00,2025-01-07 13:07:00+00:00,2025-01-11 08:52:00+00:00,NaT,0.02
2,20548,14172,11324,14235,55554,Complete,2024-04-27 15:19:37+00:00,2024-04-27 18:30:00+00:00,2024-04-28 00:06:00+00:00,NaT,0.02
3,74833,51553,40968,14235,202273,Processing,2020-12-18 00:17:49+00:00,NaT,NaT,NaT,0.02
4,80773,55668,44281,14235,218356,Processing,2023-12-17 04:19:58+00:00,NaT,NaT,NaT,0.02


In [18]:
%%bigquery --project {project_id}
select
  DISTINCT status
from
  `bigquery-public-data.thelook_ecommerce.order_items`


Query is running:   0%|          |

Downloading:   0%|          |

,status
0,Complete
1,Processing
2,Returned
3,Shipped
4,Cancelled


In [83]:
%%bigquery --project {project_id}
select
  distinct (order_id),
  created_at,
  shipped_at,
  delivered_at,
  status,
from
  `bigquery-public-data.thelook_ecommerce.order_items`
where
  status = 'Cancelled' or
  status = 'Shipped' or
  status = 'Processing'
order by
  created_at desc

Query is running:   0%|          |

Downloading:   0%|          |

,order_id,created_at,shipped_at,delivered_at,status
0,64174,2025-09-11 18:49:03.069953+00:00,2025-09-08 05:18:06.069953+00:00,NaT,Shipped
1,94915,2025-09-11 17:29:12.817900+00:00,NaT,NaT,Cancelled
2,42463,2025-09-11 17:19:50.938520+00:00,NaT,NaT,Cancelled
3,75612,2025-09-11 17:15:04.964724+00:00,2025-09-10 02:55:21.964724+00:00,NaT,Shipped
4,64174,2025-09-11 17:03:55.069953+00:00,2025-09-08 05:18:06.069953+00:00,NaT,Shipped
...,...,...,...,...,...
118806,36420,2019-01-19 00:46:57+00:00,NaT,NaT,Processing
118807,36420,2019-01-18 00:24:44+00:00,NaT,NaT,Processing
118808,36420,2019-01-16 21:34:43+00:00,NaT,NaT,Processing
118809,95818,2019-01-16 02:29:39+00:00,NaT,NaT,Processing


In [52]:
%%bigquery --project {project_id}
select
  distinct (order_id),
  created_at,
  shipped_at,
  delivered_at,
  status,
  TIMESTAMP_DIFF(delivered_at,shipped_at, HOUR) AS delivery_time,
  TIMESTAMP_DIFF(delivered_at,shipped_at, DAY) AS delivery_time_DAY
from
  `bigquery-public-data.thelook_ecommerce.order_items`
where
  created_at between '2025-03-01' and '2025-06-30' and (status = 'Complete' or  status = 'Returned') AND TIMESTAMP_DIFF(delivered_at,shipped_at, HOUR) <= 48
order by
  delivery_time desc



Query is running:   0%|          |

Downloading:   0%|          |

,order_id,created_at,shipped_at,delivered_at,status,delivery_time,delivery_time_DAY
0,27641,2025-03-28 04:44:38+00:00,2025-03-27 08:33:00+00:00,2025-03-29 09:25:00+00:00,Complete,48,2
1,93180,2025-06-04 04:24:46+00:00,2025-06-04 20:10:00+00:00,2025-06-06 20:16:00+00:00,Returned,48,2
2,106918,2025-03-29 23:04:59+00:00,2025-04-01 06:10:00+00:00,2025-04-03 06:19:00+00:00,Returned,48,2
3,23602,2025-04-13 13:23:00+00:00,2025-04-14 17:55:00+00:00,2025-04-16 18:15:00+00:00,Complete,48,2
4,80921,2025-04-23 02:49:37+00:00,2025-04-23 06:50:00+00:00,2025-04-25 07:05:00+00:00,Complete,48,2
...,...,...,...,...,...,...,...
3613,83221,2025-06-16 12:20:05+00:00,2025-06-18 04:35:00+00:00,2025-06-18 05:25:00+00:00,Returned,0,0
3614,23456,2025-06-02 09:07:09+00:00,2025-06-03 10:47:00+00:00,2025-06-03 11:01:00+00:00,Complete,0,0
3615,16256,2025-04-25 10:29:47+00:00,2025-04-26 19:08:00+00:00,2025-04-26 19:57:00+00:00,Complete,0,0
3616,79453,2025-04-17 13:21:43+00:00,2025-04-17 00:32:00+00:00,2025-04-17 00:44:00+00:00,Complete,0,0


In [77]:
%%bigquery --project {project_id}
SELECT
  COUNT(DISTINCT order_id)
FROM (
SELECT
  distinct (order_id),
  status,
  created_at,
  shipped_at,
  delivered_at,
  TIMESTAMP_DIFF(delivered_at,shipped_at, HOUR) AS delivery_time,
  TIMESTAMP_DIFF(delivered_at,shipped_at, DAY) AS delivery_time_DAY
FROM
  `bigquery-public-data.thelook_ecommerce.order_items`
WHERE
  (status = 'Complete' OR status = 'Returned')
  AND (created_at BETWEEN "2025-03-01" AND "2025-06-30")
  AND TIMESTAMP_DIFF(delivered_at, shipped_at, HOUR) < 48
)

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,2468
